In [76]:
"""
##################################################
                    SNP analysis
##################################################
"""
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', None)
import os
os.environ["ENABLE_ICICLE_GPU"] = "true"
from scipy.stats import ttest_ind

In [163]:
"""
##################################################
            Create SnpEff dataframe
##################################################
Import all text files as dataframes with same name

input: txt files from SnpEff output
output: dictionary of dataframes
"""

dataframes = {}

# Get the current directory
current_directory = os.getcwd()

# Iterate through all files in the current directory
for filename in os.listdir(current_directory):
    # Check if the file is a text file
    if filename.endswith('.txt'):
        # Extract the name of the file without the extension
        file_name_without_extension = os.path.splitext(filename)[0]
        
        # Read the text file into a DataFrame and add to dictionary dataframes
        dataframes[file_name_without_extension] = pd.read_csv(filename, sep='	', skiprows=[0])

#Only keep protein encoding genes
for dataframe in dataframes:
    dataframes[dataframe] = dataframes[dataframe][dataframes[dataframe]['BioType']=="protein_coding"]

In [242]:
"""
#####################################################################
            Create merged list of genes
#####################################################################
"""


# Assuming 'dataframes' is a dictionary containing DataFrame objects

# Get the 'GeneId' column from the 'C32' DataFrame
gene_ids = dataframes['C32']['GeneId']

# Iterate through DataFrames in the dictionary
for dataframe_name, dataframe in dataframes.items():
    # Skip 'C32' DataFrame itself
    if dataframe_name == 'C32':
        continue
    
    # Get the 'GeneId' column from the current DataFrame
    new_gene_ids = dataframe['GeneId']
#     print(new_gene_ids.nunique())
    
    # Merge the 'C32' DataFrame with the current DataFrame on 'GeneId' column
    MergedGeneIds = pd.merge(C32, new_gene_ids, how='outer')

# The result will be stored in the 'C32' DataFrame\
MergedGeneIds = MergedGeneIds['GeneId']
# MergedGeneIds.nunique()
MergedGeneIds.head()

0    PF3D7_0100100
1    PF3D7_0100200
2    PF3D7_0100300
3    PF3D7_0100400
4    PF3D7_0100600
Name: GeneId, dtype: object

In [196]:
"""
##################################################
     Create list of geneIDs, NAME and Description
##################################################
Import all text files as dataframes with same name

input: txt files from SnpEff output
output: dictionary of dataframes
"""

#import GFF file and modify chrom field to match vcf files
#Filter for protein encoding genes
df_gff = pd.read_csv("./PlasmoDB-66_Pfalciparum3D7.gff",on_bad_lines='skip', sep = '	', comment='#')

#change chromosome to numbers
replacement_dict = {
    'Pf3D7_01_v3': '1',
    'Pf3D7_02_v3': '2',
    'Pf3D7_03_v3': '3',
    'Pf3D7_04_v3': '4',
    'Pf3D7_05_v3': '5',
    'Pf3D7_06_v3': '6',
    'Pf3D7_07_v3': '7',
    'Pf3D7_08_v3': '8',
    'Pf3D7_09_v3': '9',
    'Pf3D7_10_v3': '10',
    'Pf3D7_11_v3': '11',
    'Pf3D7_12_v3': '12',
    'Pf3D7_13_v3': '13',
    'Pf3D7_14_v3': '14',
    'Pf3D7_MIT_v3': 'MIT',
    'Pf3D7_API_v3': 'API'
}

df_gff.replace(replacement_dict, inplace=True)

# df_gff.sort_values(by=['CHROM', 'START'], inplace=True)
# Only select protein coding genes 
df_gff = df_gff[df_gff['TYPE'] == 'protein_coding_gene']

# Drop the extra columns (if needed)
columns_to_drop=['DB', 'TYPE', 'X', 'Unnamed: 8', 'Y', 'CHROM', 'START', 'END', 'SCORE', 'STRAND', 'PHASE']
# columns_to_drop=['DB', 'X', 'Unnamed: 8', 'Y']
df_gff.drop(columns=columns_to_drop, inplace=True)

df_gff.head(100)
# df_gff.nunique()

,ID,NAME,DESCRIPTION
0,PF3D7_0100100,VAR,erythrocyte membrane protein 1%2C PfEMP1
4,PF3D7_0100200,RIF,rifin
8,PF3D7_0100300,VAR,erythrocyte membrane protein 1%2C PfEMP1
12,PF3D7_0100400,RIF,rifin
19,PF3D7_0100600,RIF,rifin
23,PF3D7_0100700,Plasmodium exported protein%2C unknown functio...,ebi_biotype=protein_coding
26,PF3D7_0100800,RIF,rifin
30,PF3D7_0100900,RIF,rifin
34,PF3D7_0101000,RIF,rifin
38,PF3D7_0101100,EPF4,exported protein family 4


In [331]:
"""
#############################################################################
            t-Test of controls vs drug treatment group
#############################################################################
"""

# Define exceptions to skip
#code will not work with the below list of genes for unknown reasons
exceptions = ['PF3D7_0113100', 'PF3D7_0113200', 'PF3D7_0113300', 'PF3D7_0113400','PF3D7_0113700','PF3D7_0113800','PF3D7_0113900',
             'PF3D7_0201900', 'PF3D7_0631500', 'PF3D7_0631600', 'PF3D7_0800800', 'PF3D7_0801000', 'PF3D7_1401100', 'PF3D7_1401200',
             'PF3D7_1401300', 'PF3D7_1476700', 'PF3D7_1476800', 'PF3D7_1477100', 'PF3D7_1477200', 'PF3D7_1477300', 'PF3D7_1477400',
             'PF3D7_1477800', 'PF3D7_1478000', 'PF3D7_1478100', 'PF3D7_1478600', 'PF3D7_1478800', 'PF3D7_1479000', 'PF3D7_1479100', 
             'PF3D7_0935500', 'PF3D7_0935600', 'PF3D7_0935700', 'PF3D7_0201800', 'PF3D7_0201600', 'PF3D7_0201700', 'PF3D7_0935800', 
             'PF3D7_0937100', 'PF3D7_0935900', 'PF3D7_0936000', 'PF3D7_0936100', 'PF3D7_0936200', 'PF3D7_0936300', 'PF3D7_0936400',
             'PF3D7_0936500', 'PF3D7_0936600', 'PF3D7_0936700', 'PF3D7_0936800', 'PF3D7_0937000']

# Define control groups and drug treatment groups
control_group_names = ['C32', 'C40', 'C46', 'CB']
treatment_group_names = ['T106', 'T112', 'T132', 'T215', 'T219', 'T224', 'T301', 'T305', 'T307']

# Define variant types
variant_types = [
    'variants_impact_HIGH',
    'variants_impact_LOW',
    'variants_impact_MODERATE',
    'variants_impact_MODIFIER',
    'variants_effect_conservative_inframe_deletion',
    'variants_effect_conservative_inframe_insertion',
    'variants_effect_disruptive_inframe_deletion',
    'variants_effect_disruptive_inframe_insertion',
    'variants_effect_downstream_gene_variant',
    'variants_effect_frameshift_variant',
    'variants_effect_intron_variant',
    'variants_effect_missense_variant',
    'variants_effect_non_coding_transcript_exon_variant',
    'variants_effect_splice_acceptor_variant',
    'variants_effect_splice_donor_variant',
    'variants_effect_splice_region_variant',
    'variants_effect_start_lost',
    'variants_effect_stop_gained',
    'variants_effect_stop_lost',
    'variants_effect_stop_retained_variant',
    'variants_effect_synonymous_variant',
    'variants_effect_upstream_gene_variant'
]

# Perform statistical analysis for each gene and variant type

#Collect results here
results = []

#Define control_df which is used to iterate through unique geneIDs
control_df = MergedGeneIds

# Flag to indicate when to start processing
# start_processing = False

#Loop through each geneID and every variant type, get list of control and treatment group, do ttest
for geneID in control_df:
    
#     # Start processing when the geneID matches the specified geneID
#     if geneID == exceptions[-1]:
#         start_processing = True
    
#     # Skip geneID if it's in the exceptions list or before the specified geneID
#     if not start_processing or geneID in exceptions:
#         continue
    
    #print geneID for tracking
#     print(geneID)
    
    if geneID in exceptions:
        continue
    
    for variant_type in variant_types:
        
        control_counts = [dataframes[group_name].loc[dataframes[group_name]['GeneId'] == geneID, variant_type].values[0] for group_name in control_group_names]
        
        treatment_counts = [dataframes[group_name].loc[dataframes[group_name]['GeneId'] == geneID, variant_type].values[0] for group_name in treatment_group_names]
        
        # Perform the t-test
        t_statistic, p_value = ttest_ind(control_counts, treatment_counts)
        
        # Append results to the results list if p-value is less than 0.05
        if p_value < 0.05:
            results.append({'GeneId': geneID, 'Variant_type': variant_type, 'p_value': p_value})

results_df = pd.DataFrame(results)

print(results_df)

C:\Users\khang\AppData\Local\Temp\ipykernel_28828\3827325168.py:82: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  t_statistic, p_value = ttest_ind(control_counts, treatment_counts)


                  GeneId                                    Variant_type  \
0          PF3D7_0100100                            variants_impact_HIGH   
1          PF3D7_0100100              variants_effect_frameshift_variant   
2          PF3D7_0100100                     variants_effect_stop_gained   
3          PF3D7_0100200                     variants_effect_stop_gained   
4          PF3D7_0100300                            variants_impact_HIGH   
5          PF3D7_0100300    variants_effect_disruptive_inframe_insertion   
6          PF3D7_0100300              variants_effect_frameshift_variant   
7          PF3D7_0100300           variants_effect_upstream_gene_variant   
8          PF3D7_0100400                        variants_impact_MODIFIER   
9          PF3D7_0100400           variants_effect_upstream_gene_variant   
10         PF3D7_0100800                            variants_impact_HIGH   
11         PF3D7_0100800   variants_effect_conservative_inframe_deletion   
12         P

In [334]:
"""
#############################################################################
            Annotations added to results dataframe
#############################################################################
"""
results_df2=results_df 
# Add stars to p values
def generate_p_value_star(p_value):
    if p_value < 0.0005:
        return '***'
    elif p_value < 0.005:
        return '**'
    else:
        return '*'

# Apply the function to create the p_value* column
results_df2['p_value_star'] = results_df2['p_value'].apply(generate_p_value_star)

#Merge with df_gff for gene names
annotated_results_df2 = pd.merge(results_df2, df_gff, left_on='GeneId', right_on='ID', how='left')
annotated_results_df2 = annotated_results_df2[['GeneId', 'NAME', 'DESCRIPTION', 'Variant_type', 'p_value', 'p_value_star']]


print(annotated_results_df2)
# annotated_results_df.to_csv('annotated_results_df.csv')

                  GeneId                                               NAME  \
0          PF3D7_0100100                                                VAR   
1          PF3D7_0100100                                                VAR   
2          PF3D7_0100100                                                VAR   
3          PF3D7_0100200                                                RIF   
4          PF3D7_0100300                                                VAR   
5          PF3D7_0100300                                                VAR   
6          PF3D7_0100300                                                VAR   
7          PF3D7_0100300                                                VAR   
8          PF3D7_0100400                                                RIF   
9          PF3D7_0100400                                                RIF   
10         PF3D7_0100800                                                RIF   
11         PF3D7_0100800                            

In [343]:
"""
##################################################
            Drop VAR and RIF
##################################################

"""
df2 = annotated_results_df2

df3 = df2.drop(df2[df2.NAME =="VAR" ].index)
df4 = df3.drop(df3[df3.NAME =="RIF"].index)
df5 = df4.drop(df4[df4.NAME =="stevor"].index)
# df5.head()
# df5.nunique()
# df5.to_csv('C_v_T_annotated_results_df_dropped_VAR_RIF_stevor.csv')
df5.nunique()

GeneId          942
NAME            602
DESCRIPTION     360
Variant_type     19
p_value         701
p_value_star      3
dtype: int64

In [348]:
"""
##################################################
            Add Impacts to Effects
##################################################

"""


#change variant effects
df6 = df5
replacement_dict = {
    'variants_effect_splice_region_variant': 'LOW',
    'variants_effect_upstream_gene_variant': 'MODIFIER',
    'variants_effect_downstream_gene_variant': 'MODIFIER',
    'variants_effect_intron_variant': 'MODIFIER',
    'variants_effect_missense_variant': 'MODERATE',
    'variants_effect_synonymous_variant': 'LOW',
    'variants_effect_disruptive_inframe_deletion': 'MODERATE', 
    'variants_effect_frameshift_variant': 'HIGH',
    'variants_effect_stop_gained': 'HIGH', 
    'variants_effect_start_lost': 'HIGH',
    'variants_effect_conservative_inframe_insertion': 'MODERATE',
    'variants_effect_stop_retained_variant': 'LOW',
    'variants_effect_disruptive_inframe_insertion': 'MODERATE',
    'variants_effect_conservative_inframe_deletion': 'MODERATE',
    'variants_effect_splice_acceptor_variant': 'HIGH'
}

# Add a new column 'Impact' to df5 by mapping the values from 'Variant_type' to 'replacement_dict'
df6['Impact'] = df6['Variant_type'].map(replacement_dict)

df6.head()
df6.to_csv('C_v_T_annotated_results_df_dropped_VAR_RIF_stevor_+Impact.csv')